In [207]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [208]:
full_data = []
num_of_data = 2000


In [209]:
def too_much(table, offset=0):
    for i, data in enumerate(table.find_all('tr')[1:], 1):
        temp = data.text.split('$')

        number = str(i + offset)
        if len(number) > 3:
            list_number = list(number)
            list_number.insert(len(number) - 3, ',')
            number = ''.join(list_number)

        temp[0] = temp[0].replace(number, '')
        temp = [i + offset] + temp

        temp[3] = temp[3].replace('<', '')

        if len(temp) == 4:
            temp[2] = temp[2].replace('-', '')
            temp.insert(3, '0,00000%')

        index = len(temp[3]) - temp[3][::-1].find(',') + 2
        temp.insert(3, temp[3][:index + 1])
        temp.insert(4, temp[4][index + 1:])
        temp.pop(5)
        temp.append(temp[-1][-4:])
        temp[5] = temp[5][:-4]

        index2 = len(temp[5]) - temp[5][::-1].find(',') + 2
        temp.insert(5, temp[5][:index2 + 1])
        temp.insert(6, temp[6][index2 + 1:])
        temp.pop(7)

        if '-' in temp[5]:
            temp[5] = temp[5][:-1]
            temp[6] = '0'

        temp[6] = temp[6].replace('<', '')

        full_data.append(temp)


In [210]:
text = ''

for j in range(0, num_of_data + 1, 200):
    with open(f'data/worldwide_gross{j}.txt', 'r', encoding='utf-8') as file:
        text += file.read()


In [211]:
soup = BeautifulSoup(text, 'html.parser')


In [212]:
table = soup.find_all('table')

for j in range(num_of_data // 200):
    too_much(table[j], 200*j)


In [213]:
df = pd.DataFrame(full_data)
df.columns = ['Rank', 'Title', 'Worldwide_gross', 'Domestic_gross', 'Domestic %', 'Foreign_gross', 'Foreign %', 'Year']
df
df.to_excel('data.xlsx', index=False)

In [214]:
df.Worldwide_gross = df.Worldwide_gross.str.replace(',', '')
df.Worldwide_gross = df.Worldwide_gross.astype(np.int64)

df.Domestic_gross = df.Domestic_gross.str.replace(',', '')
df.Domestic_gross = df.Domestic_gross.astype(np.int64)

df['Domestic %'] = df['Domestic %'].str.replace('-', '0')
df['Domestic %'] = df['Domestic %'].str.replace('%', '').astype(float)

df.Foreign_gross = df.Foreign_gross.str.replace(',', '')
df.Foreign_gross = df.Foreign_gross.astype(np.int64)

df['Foreign %'] = df['Foreign %'].str.replace('-', '0')
df['Foreign %'] = df['Foreign %'].str.replace('%', '').astype(float)

df.Year = df.Year.astype(int)


In [215]:
df.to_excel('data.xlsx', index=False)
